<a href="https://colab.research.google.com/github/Nekoiii/ML_Practices_colab/blob/main/Lora_test_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sd-scripts github: https://github.com/kohya-ss/sd-scripts

Reference article：
· https://tkstock.site/2023/05/19/googlecolab-stablediffusion-lora-finetuning/#i-7

· https://www.ipentec.com/document/software-stable-diffusion-lora-learn#section_12



In [1]:
#@title Mount Google Drive
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [9]:
sd_path='/content/drive/MyDrive/StableDifussion/sd-scripts'
#lora_path=
pretrained_model_name_or_path='/content/drive/MyDrive/StableDifussion/models/anything-v4.5-pruned-fp16.ckpt'


In [4]:
#@title Clone sd-script
%%script false --no-raise-error
%%capture
!git clone https://github.com/kohya-ss/sd-scripts '{sd_path}'

In [10]:
#@title Install sd-script Requirements

#According to the sd-scripts github documentation, we should install PyTorch before install requirements.txt 
#because the versions of it depends on your environment and requirements.txt does not contain requirements for PyTorch.

#%%capture

%cd '{sd_path}'
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install  --upgrade -r requirements.txt
#!pip install -U --pre triton
!pip install xformers==0.0.16rc425
%cd /content

/content/drive/MyDrive/StableDifussion/sd-scripts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/StableDifussion/sd-scripts
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.1/503.1 kB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content


In [ ]:
#@title Use fp16

# Change some config (Some user reports [ValueError: fp16 mixed precision requires a GPU] is occurred in training )

%%capture

!accelerate config default --mixed_precision fp16

In [14]:
# Copy Training Images Folder
training_data_path='/content/training_data'
original_data_path='/content/drive/MyDrive/datasets/imgs/resized_canbright_imgs_1_partial'

%cp -r '{original_data_path}' '{training_data_path}'

In [ ]:
# Generate captions and tags for each image.
%%capture

%cd '{sd_path}'

!python finetune/make_captions.py --batch_size 8 '{training_data_path}'
!python finetune/tag_images_by_wd14_tagger.py --batch_size 4 '{training_data_path}'

In [ ]:
# Create Descriptions For Images

# Merges captions with metadata and outputs the result in meta_cap.json.
# Merges Data Description (DD) tags with metadata from the previous step and generates a new file meta_cap_dd.json.
# Cleans and preprocesses the captions and tags from the previous step and generates a cleaned version in meta_clean.json.

%%capture

!python finetune/merge_captions_to_metadata.py '{training_data_path}' meta_cap.json
!python finetune/merge_dd_tags_to_metadata.py '{training_data_path}' --in_json meta_cap.json meta_cap_dd.json
!python finetune/clean_captions_and_tags.py meta_cap_dd.json meta_clean.json

In [ ]:

%%capture

!python finetune/prepare_buckets_latents.py \
'{training_data_path}' meta_clean.json meta_lat.json \
'{pretrained_model_name_or_path}' \
--batch_size 4 \
--max_resolution 512,512 \
--mixed_precision no

In [13]:
!pwd

/content/sd-scripts


In [ ]:
#train_network.py: https://github.com/kohya-ss/sd-scripts/blob/main/train_network.py

!accelerate launch  \
--num_cpu_threads_per_process 1 \
train_network.py \
--pretrained_model_name_or_path='{pretrained_model_name_or_path}' \
--in_json modified_meta_lat.json \
--train_data_dir='{training_data_path}' \
--output_dir=output_models \
--shuffle_caption \
--train_batch_size=1 \
--learning_rate=1e-4 \
--max_train_steps=500 \
--use_8bit_adam \
--xformers --gradient_checkpointing \
--mixed_precision=fp16 \
--save_every_n_epochs=10 \
--save_precision=fp16 \
--network_module=networks.lora\
--training_comment="cb_satoo" \
#--save_model_as= safetensors
#--save_model_as= pt 
#--output_name=satoo_with_descriptions \


In [31]:
output_model_path='/content/drive/MyDrive/StableDifussion/sd-scripts/output_models/last.safetensors'
save_path='/content/drive/MyDrive/my_models/canbright_satoo/'
new_file_name = 'satoo_without_descriptions_5000.safetensors'

!mkdir -p '{save_path}'
!cp '{output_model_path}' '{save_path}/{new_file_name}'

In [ ]:
# https://github.com/kohya-ss/sd-scripts/blob/main/gen_img_diffusers.py

used_model='/content/drive/MyDrive/my_models/canbright_satoo/satoo_with_descriptions_10000.safetensors'
steps=20
seed=1
prompt='a boy,((((cb_satoo))))'

outdir='/content/output_imgs/'

!python /content/drive/MyDrive/StableDifussion/sd-scripts/gen_img_diffusers.py \
  --ckpt '{pretrained_model_name_or_path}' \
  --n_iter 1 \
  --scale 7.5 \
  --steps '{steps}' \
  --outdir  '{outdir}' \
  --prompt '{prompt}' \
  --xformers \
  --W 512 \
  --H 512 \
  --seed  '{seed}' \
  --sampler k_euler_a \
  --network_module networks.lora \
  --network_weights  '{used_model}' \
  --network_mul 1 \
  --max_embeddings_multiples 3 \
  --clip_skip 1 \
  --batch_size 1 \
  --images_per_prompt 1 
  #--bf16 \
  #--seed   1 \

In [27]:
# *for test: remove "caption" and "tags" 

import json

with open('/content/drive/MyDrive/StableDifussion/sd-scripts/meta_lat.json') as f:
    data = json.load(f)

for key in data:
    if "caption" in data[key]:
        del data[key]["caption"]
    if "tags" in data[key]:
        del data[key]["tags"]

with open('/content/drive/MyDrive/StableDifussion/sd-scripts/modified_meta_lat.json', 'w') as f:
    json.dump(data, f)
